#### **But DU MODELE:**
#### **Prédire et prévenir les blessures causées lors de matchs de tennis.**

In [ ]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Chargement des données
chemin_excel_blessures = "../DataSets/BlessuresTennis_Causes.xlsx"
df = pd.read_excel(chemin_excel_blessures)

In [ ]:
# Nettoyage des données
def fix_separators(df):
    corrections = {}
    for column in df.columns:
        corrections[column] = []
        for i, value in enumerate(df[column]):
            if pd.notnull(value) and '; ' in str(value):
                parts = str(value).split('; ')
                if len(parts) > 1:
                    df.at[i, column] = '; '.join(part.strip() for part in parts)
                    corrections[column].append(f"Ligne {i+1}: Séparateur corrigé")
            elif pd.notnull(value) and ';' in str(value):
                df.at[i, column] = '; '.join(part.strip() for part in str(value).split(';') if part.strip() != '')
                corrections[column].append(f"Ligne {i+1}: Séparateur corrigé")
    return df, corrections

df, corrections = fix_separators(df[['ID', 'CategBlessure', 'TypeBlessure', 'Blessure', 'Symptomes_Blessures', 'Traitement_Blessures', 'Prescriptions_Blessures', 'Prix_Prescriptions (en € )', 'Images_Prescriptions', 'Descriptifs_Prescriptions', 'LienAchat_Prescriptions', 'Causes_Blessure', 'Zone_Blessures', 'Description_Zone']])
print("Séparateurs corrigés :")
for column, correction in corrections.items():
    if correction:
        print(f"Colonne {column}:")
        for item in correction:
            print(f"- {item}")
        print('\n')

In [ ]:
# Fonction pour extraire les valeurs numériques
def extract_float(value):
    matches = re.findall(r'\d+\.\d+', str(value)) # type: ignore
    if matches:
        return float(matches[0])
    else:
        return None

In [ ]:
# Remplacer les virgules par des points dans les valeurs de la colonne 'Prix_Prescriptions (en € )'
df['Prix_Prescriptions (en € )'] = df['Prix_Prescriptions (en € )'].str.replace(',', '.')

# Extraire les valeurs numériques de la colonne et les convertir en flottant
df['Prix_Prescriptions (en € )'] = df['Prix_Prescriptions (en € )'].apply(extract_float)

# Supprimer les lignes avec des valeurs non numériques
df = df.dropna(subset=['Prix_Prescriptions (en € )'])

In [ ]:
# Maintenant, vous pouvez continuer avec votre code d'entraînement de modèle

columns_to_convert = ['Prescriptions_Blessures', 'Prix_Prescriptions (en € )', 'Images_Prescriptions', 'Descriptifs_Prescriptions', 'LienAchat_Prescriptions']
df = df.dropna(subset=columns_to_convert, how='all')
df.head()

In [ ]:
def split_columns(row):
    try:
        prescriptions = row['Prescriptions_Blessures'].split('; ')
        prices = [float(price.replace(',', '.')) for price in row['Prix_Prescriptions (en € )'].split('; ')]
        images = row['Images_Prescriptions'].split('; ')
        descriptions = row['Descriptifs_Prescriptions'].split('; ')
        links = row['LienAchat_Prescriptions'].split('; ')
        return list(zip(prescriptions, prices, images, descriptions, links))
    except AttributeError:
        return [(row['Prescriptions_Blessures'], row['Prix_Prescriptions (en € )'], row['Images_Prescriptions'], row['Descriptifs_Prescriptions'], row['LienAchat_Prescriptions'])]

prescriptions_df = df.apply(split_columns, axis=1).explode().apply(lambda x: pd.Series(x, index=['Prescription', 'Prix', 'Image', 'Descriptif', 'Lien d\'achat']))

print("\nPrescriptions :")
for index, row in prescriptions_df.iterrows():
    print(f"| Prescription : {row['Prescription']} | Prix : {row['Prix']} | Image : {row['Image']} | Descriptif : {row['Descriptif']} | Lien d'achat : {row['Lien d\'achat']} |")

In [ ]:
non_prescriptions_columns = ['Traitement_Blessures', 'Causes_Blessure', 'Zone_Blessures']
non_prescriptions_df = df[non_prescriptions_columns].reset_index().rename(columns={'index': 'Blessure'})
for col in non_prescriptions_columns:
    non_prescriptions_df[col] = non_prescriptions_df[col].str.split('; ')
non_prescriptions_df = non_prescriptions_df.explode('Traitement_Blessures').explode('Causes_Blessure').explode('Zone_Blessures')

print("\nDonnées pour le traitement, les causes et la zone des blessures :")
print(non_prescriptions_df)

In [ ]:
# Traitement des données
# (Vous pouvez ajouter ici le traitement des données sur les prescriptions médicales, la séparation des colonnes, etc.)

# Encodage et normalisation des données
categorical_columns = ['CategBlessure', 'TypeBlessure', 'Blessure', 'Symptomes_Blessures', 
                       'Traitement_Blessures', 'Prescriptions_Blessures', 'Causes_Blessure', 
                       'Zone_Blessures', 'Description_Zone']
df[categorical_columns] = df[categorical_columns].fillna('')
df[categorical_columns] = df[categorical_columns].astype(str)

encoder = OneHotEncoder()
encoded_categorical = encoder.fit_transform(df[categorical_columns])

scaled_numerical = np.array([price for price in df['Prix_Prescriptions (en € )']]).reshape(-1, 1)

X = np.concatenate((encoded_categorical.toarray(), scaled_numerical), axis=1)
y = df['Blessure']

In [ ]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialisation et entraînement du modèle
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Prédiction sur les données de test
y_pred = model.predict(X_test)

In [ ]:
# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

In [ ]:
# Sélection des colonnes pertinentes pour la régression
X_reg = df[['CategBlessure', 'TypeBlessure', 'Blessure', 'Traitement_Blessures', 'Prescriptions_Blessures']]
y_reg = df['Prix_Prescriptions (en € )']  

# Encodage one-hot des colonnes catégorielles
X_reg_encoded = pd.get_dummies(X_reg)

# Division des données en ensembles d'entraînement et de test
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg_encoded, y_reg, test_size=0.2, random_state=42)

# Initialisation et entraînement du modèle de régression linéaire
regression_model = LinearRegression()
regression_model.fit(X_train_reg, y_train_reg)

# Prédiction sur les données de test
y_pred_reg = regression_model.predict(X_test_reg)

# Évaluation du modèle
mse_reg = mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print("Mean Squared Error (Regression):", mse_reg)
print("R^2 Score (Regression):", r2_reg)

In [ ]:
# Sélection de la première observation du jeu de données de test
observation_index = 0  # Index de l'observation dans le jeu de données de test
observation_features = X_test[observation_index]  # Caractéristiques de l'observation
observation_price = y_test[observation_index]  # Prix réel de prescription pour cette observation

# Prédiction du prix de prescription pour cette observation
predicted_price = model.predict([observation_features])

# Affichage des résultats
print("Observation - Caractéristiques:", observation_features)
print("Prix réel de prescription:", observation_price)
print("Prédiction du prix de prescription:", predicted_price)